# Emoji Prediction

In [39]:
# !pip install emoji

In [40]:
import emoji
import numpy as np
import pandas as pd

In [41]:
mapping = pd.read_csv("../input/twitter-emoji-prediction/Mapping.csv")
output = pd.read_csv("../input/twitter-emoji-prediction/OutputFormat.csv")
train = pd.read_csv("../input/twitter-emoji-prediction/Train.csv")
test = pd.read_csv("../input/twitter-emoji-prediction/Test.csv")

In [42]:
mapping

In [43]:
mapping = mapping.drop("Unnamed: 0", axis=1)
mapping.head()

In [44]:
mapping = mapping.set_index('number')

In [45]:
mapping.head()

In [46]:
mapping_dict = mapping.to_dict()
mapping_dict = mapping_dict['emoticons']
mapping_dict

In [47]:
train.head()

In [48]:
train.columns

In [49]:
train = train.drop('Unnamed: 0', axis=1)

In [50]:
train.tail()

In [51]:
# changing the labels to emoji
# train = train.replace({"Label": mapping_dict})

In [52]:
train.head()

## Text Preprocessing using Embeddings

In [53]:
X_train = train['TEXT'].values
y_train = train['Label'].values

print(X_train.shape, y_train.shape)

In [54]:
# maxLen = X_train.map(len).max()

In [55]:
# maxLen, type(maxLen)

In [56]:
file = open("../input/glove6b50dtxt/glove.6B.50d.txt", encoding = 'utf8')

In [57]:
file

In [58]:
def intialize_emb_matrix(file):
    embedding_matrix = {}
    for line in file:
        values = line.split()
        word = values[0]
        embedding = np.array(values[1:], dtype='float64')
        embedding_matrix[word] = embedding

    return embedding_matrix 


In [59]:
embedding_matrix = intialize_emb_matrix(file)

In [60]:
embedding_matrix['dance']

In [61]:
embedding_matrix['dance'].shape 

In [62]:
def get_emb_data(data, max_len):
#     max_len = 168
    embedding_data = np.zeros((len(data), max_len, 50))  # from glove6B50d
    
    for idx in range(data.shape[0]):
        words_in_sentence = data[idx].split()
        
        for i in range(len(words_in_sentence)):
            if embedding_matrix.get(words_in_sentence[i].lower()) is not None:
                embedding_data[idx][i] = embedding_matrix[words_in_sentence[i].lower()]
                
    return embedding_data

In [63]:
# representing each word in the sentence acc. to glove embedding [str --> numerical]
X_temb = get_emb_data(X_train, 168)

In [64]:
X_temb

In [65]:
X_temb.shape

In [66]:
# X_temb = X_temb.reshape(-1, 20, 50)
# X_temb.shape

In [67]:
# convert outputs to categorical variables
from keras.utils.np_utils import to_categorical

In [68]:
# converting y_train to one hot vectors so that cross-entropy loss can be used.
y_train = to_categorical(y_train)

In [69]:
y_train

## Model Building

In [70]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, Dense, Dropout

In [71]:
model = Sequential()

In [72]:
lm = len(mapping_dict)
lm

In [73]:
model.add(LSTM(units = 256, return_sequences=True, input_shape = (168,50)))
model.add(Dropout(0.3))
model.add(LSTM(units=128))
model.add(Dropout(0.3))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=20, activation='softmax'))

In [74]:
model.summary()

In [75]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['acc'])

In [76]:
# model Training
res = model.fit(X_temb, y_train, validation_split=0.2, batch_size=32, epochs=10, verbose=2)

## Using Another dataset for Text to emoji prediction

In [77]:
train_data = pd.read_csv("../input/emojify/train_emoji.csv", header=None)
test_data = pd.read_csv("../input/emojify/test_emoji.csv", header=None)

In [78]:
train_data.head()

In [79]:
test_data.head()

In [80]:
train_data.drop(labels=[2, 3], axis=1, inplace=True)

In [81]:
train_data.tail()

In [82]:
emoji.EMOJI_UNICODE['en']

In [83]:
emoji.emojize(':angry_face:')

In [84]:
emoji_mapping = {
    '0': ':beating_heart:',
    '1': ':baseball:',
    '2': ':beaming_face_with_smiling_eyes:',
    '3': ':angry_face:',
    '4': ':face_savoring_food:'
}

In [85]:
for key, value in emoji_mapping.items():
    emoji_mapping[key] = emoji.emojize(value)
    
print(emoji_mapping)
    

In [86]:
X_train = train_data[0].values
y_train = train_data[1].values

In [87]:
# def intialize_emb_matrix(file):
#     embedding_matrix = {}
#     for line in file:
#         values = line.split()
#         word = values[0]
#         embedding = np.array(values[1:], dtype='float64')
#         embedding_matrix[word] = embedding

#     return embedding_matrix 

file

In [88]:
# we have already created an embedding matrix using the golve vectors
embedding_matrix

In [89]:
X_train = get_emb_data(X_train, 10)

In [90]:
X_train

In [91]:
y_train = to_categorical(y_train)
y_train

In [92]:
model_tte = Sequential()
model_tte.add(LSTM(units=64, input_shape=(10, 50), return_sequences=True))
model_tte.add(Dropout(0.3))
model_tte.add(LSTM(units=32))
model_tte.add(Dropout(0.2))
model_tte.add(Dense(units=10, activation='relu'))
model_tte.add(Dense(units=5, activation='softmax'))

In [93]:
model_tte.summary()

In [94]:
model_tte.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['acc'])

In [95]:
history = model_tte.fit(X_train, y_train, validation_split=0.1, verbose=2, batch_size=32, epochs=100)

In [96]:
model_tte2 = Sequential()
model_tte2.add(LSTM(units=128, input_shape=(10, 50), return_sequences=True))
model_tte2.add(Dropout(0.3))
model_tte2.add(LSTM(units=64))
model_tte2.add(Dropout(0.2))
model_tte2.add(Dense(units=32, activation='relu'))
model_tte2.add(Dense(units=20, activation='relu'))
model_tte2.add(Dense(units=5, activation='relu'))

In [97]:
model_tte2.summary()

In [98]:
model_tte2.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['acc'])

In [99]:
history2 = model_tte2.fit(X_train, y_train, validation_split=0.1, verbose=2, batch_size=32, epochs=100)

In [100]:
model_tte3 = Sequential()
model_tte3.add(LSTM(units=128, input_shape=(10, 50), return_sequences=True))
model_tte3.add(Dropout(0.3))
model_tte3.add(LSTM(units=64))
model_tte3.add(Dropout(0.2))
model_tte3.add(Dense(units=32, activation='relu'))
model_tte3.add(Dense(units=20, activation='relu'))
model_tte3.add(Dense(units=5, activation='softmax'))

In [101]:
model_tte3.summary()

In [102]:
model_tte3.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['acc'])

In [103]:
history3 = model_tte3.fit(X_train, y_train, validation_split=0.1, verbose=2, batch_size=32, epochs=100)